Elina Fetingyte 1712325

---


SSD, IoU balanced loss, Human body/Horse/Knife


---



Import dependencies

In [12]:
import os
import shutil
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image

import torch
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from google.colab import drive

Import model from drive

In [11]:
# !rm -rf ./SSD/
drive.mount('/content/drive')
!mkdir model
!cp ./drive/MyDrive/SSD/model* ./model/

from model.ssd import SSD
from model.utils import generate_dboxes, Encoder, colors
from model.loss import Loss
from model.process import train, evaluate
from model.dataset import collate_fn, OIDataset
from model.transform import SimpleTransformer

Set parameters for training

In [ ]:
epochs = 10
batch_size = 4
multistep = [5, 7]
learning_rate = 2.6e-3
mns_threshold = 0.5
ioub_loss = True

drive_path = '/content/drive/MyDrive/SSD'
checkpoint_path = os.path.join(drive_path, "/trained/SSD.pth")
log_path = os.path.join(drive_path, "logs")

Prepare data and model

In [ ]:
if torch.cuda.is_available():
    print('Will compute using CUDA')
torch.cuda.manual_seed(123)

train_params = {"batch_size": batch_size,
                "shuffle": True,
                "drop_last": False,
                "num_workers": 4,
                "collate_fn": collate_fn}

test_params = {"batch_size": batch_size,
               "shuffle": True,
               "drop_last": False,
               "num_workers": 4,
               "collate_fn": collate_fn}
            
dboxes = generate_dboxes()
model = SSD()
train_set = OIDataset(SimpleTransformer(dboxes))
train_loader = DataLoader(train_set, **train_params)
test_set = OIDataset(SimpleTransformer(dboxes, eval=True), train=False)
test_loader = DataLoader(test_set, **test_params)

encoder = Encoder(dboxes)

learning_rate = learning_rate * (batch_size / 32)
criterion = Loss(dboxes, use_weighted_iou=ioub_loss)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9,
                            weight_decay=0.0005,
                            nesterov=True)
scheduler = MultiStepLR(optimizer=optimizer, milestones=multistep, gamma=0.1)

if torch.cuda.is_available():
    model.cuda()
    criterion.cuda()

model = torch.nn.DataParallel(model)
writer = SummaryWriter(log_path)

Load checkpoint if possible

In [ ]:
if os.path.isfile(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    first_epoch = checkpoint["epoch"] + 1
    model.module.load_state_dict(checkpoint["model_state_dict"])
    scheduler.load_state_dict(checkpoint["scheduler"])
    optimizer.load_state_dict(checkpoint["optimizer"])
else:
    first_epoch = 0

Train

In [ ]:
for epoch in range(first_epoch, epochs):
    train(model, train_loader, epoch, writer, criterion, optimizer, scheduler)
    evaluate(model, test_loader, encoder, nms_threshold)

    checkpoint = {"epoch": epoch,
                  "model_state_dict": model.module.state_dict(),
                  "optimizer": optimizer.state_dict(),
                  "scheduler": scheduler.state_dict()}
    torch.save(checkpoint, checkpoint_path)

Draw predictions

In [ ]:
classes = ['Background', 'Knife', 'Horse', 'Human']
cls_threshold = 0.3
nms_threshold = 0.5
model_path = f'{drive_path}/trained/SSD.pth'

def ensure_legal(xmin, ymin, xmax, ymax, width, height):
    if xmin < 0:
        xmin = 0
    if ymin < 0:
        ymin = 0
    if xmax > width:
        xmax = width
    if ymax > height:
        ymax = height
    return xmin, ymin, xmax, ymax


def test_one(path):
    model = SSD()
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint["model_state_dict"])
    if torch.cuda.is_available():
        model.cuda()
    model.eval()
    dboxes = generate_dboxes()
    transformer = SimpleTransformer(dboxes, eval=True)
    img = Image.open(path).convert("RGB")
    img, _, _ = transformer(img, torch.zeros(4), torch.zeros(1))
    encoder = Encoder(dboxes)

    if torch.cuda.is_available():
        img = img.cuda()

    with torch.no_grad():
        ploc, plabel = model(img.unsqueeze(dim=0))
        result = encoder.decode_batch(ploc, plabel, nms_threshold, 20)[0]
        loc, label, prob = [r.cpu().numpy() for r in result]
        best = np.argwhere(prob > cls_threshold).squeeze(axis=1)
        loc = loc[best]
        label = label[best]
        prob = prob[best]
        output_img = cv2.imread(path)

        if len(loc) > 0:
            height, width, _ = output_img.shape
            loc[:, 0::2] *= width
            loc[:, 1::2] *= height
            loc = loc.astype(np.int32)

            for box, lb, pr in zip(loc, label, prob):
                category = classes[lb]
                color = colors[lb]
                xmin, ymin, xmax, ymax = box
                xmin, ymin, xmax, ymax = ensure_legal(xmin, ymin, xmax, ymax, width, height)
                cv2.rectangle(output_img, (xmin, ymin), (xmax, ymax), color, 2)
                text_size = cv2.getTextSize(category + " : %.2f" % pr, cv2.FONT_HERSHEY_PLAIN, 1, 1)[0]
                cv2.rectangle(output_img, (xmin, ymin), (xmin + text_size[0] + 3, ymin + text_size[1] + 4), color, -1)
                cv2.putText(output_img, category + " : %.2f" % pr,
                    (xmin, ymin + text_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1,
                    (255, 255, 255), 1)
        cv2_imshow(output_img)

In [ ]:
import random

input_folder = f'{drive_path}/Dataset_lim/validation/horse/'
for image in random.choices(os.listdir(input_folder), k=3):
    test_one(input_folder + image)
input_folder = f'{drive_path}/Dataset_lim/validation/human_body/'
for image in random.choices(os.listdir(input_folder), k=3):
    test_one(input_folder + image)
input_folder = f'{drive_path}/Dataset_lim/validation/knife/'
for image in random.choices(os.listdir(input_folder), k=3):
    test_one(input_folder + image)